In [1]:
import regex as re
import pandas as pd
import random
import requests
import urllib.request
from IPython.display import Image

#api de google sheets
from __future__ import print_function
import os.path
from bs4 import BeautifulSoup
import lxml.html
try:
    from googleapiclient.discovery import build
    from google_auth_oauthlib.flow import InstalledAppFlow
    from google.auth.transport.requests import Request
    from google.oauth2.credentials import Credentials
except:
    import sys
    !{sys.executable} -m pip install --upgrade google-api-python-client google-auth-httplib2 google-auth-oauthlib
    from googleapiclient.discovery import build
    from google_auth_oauthlib.flow import InstalledAppFlow
    from google.auth.transport.requests import Request
    from google.oauth2.credentials import Credentials

# display de imatges    
try:
    from image import DrawImage
except:
    import sys
    !{sys.executable} -m pip install terminal-img
    from image import DrawImage

In [2]:
class Recomanador:
    
    def __init__(self):
        self.players = None
        self.dificultat = None
        self.dif_min = None
        self.dif_max = None
        self.temps = None
        
        
    
    def simple_questions(self):
        '''pregunta als jugadors: quants son, quina dificultat i quant de temps volen jugar'''
        while True:
            valid = False
            while valid == False:
                self.players = input('quants jugadors sou? ')
                if self.players.isnumeric():
                    self.players = int(self.players)
                    valid = True
            break

        while True:
            valid = False
            while valid == False:
                print('quina profunditat de joc esperes? Facil/Mitja/Dificil \n')
                self.dificultat = input('introdueix F,M o D ')
                if self.dificultat.upper() == 'F':
                    self.dificultat = self.dificultat.upper()
                    valid = True
                elif self.dificultat.upper() == 'M':
                    self.dificultat = self.dificultat.upper()
                    valid = True
                elif self.dificultat.upper() == 'D':
                    self.dificultat = self.dificultat.upper()
                    valid = True
            break

        if self.dificultat == 'F':
            self.dif_min = 0
            self.dif_max = 2.5
        elif self.dificultat == 'M':
            self.dif_min = 2
            self.dif_max = 3.5
        else:
            self.dif_min = 3.5
            self.dif_max = 5    

        while True:
            valid = False
            while valid == False:
                print('quanta estona voleu jugar aproximadament? ')
                self.temps = input('introdueix temps en minuts ')
                if self.temps.isnumeric():
                    self.temps = int(self.temps)
                    valid = True
            break
    
    
    def simple_choice(self,data):
        '''selecciona un joc, mostra una imatge si pot, i pregunta si va bé el joc seleccionat'''
        queried_jocs = data.query('min <= @self.players & max >= @self.players & Duració <= (@self.temps + 20) & Duració >= (@self.temps - 20) & Profunditat >= @self.dif_min & Profunditat <= @self.dif_max ')
        while True:
            
            # seleccionar juego al azar según criterios
            try:
                pick = random.choice(list(queried_jocs['Nom del joc']))
            except:
                print('no hi ha cap joc que cumpleixi els teus requisits\n')
                print("et recomanaré un joc a l'atzar\n")
                pick = random.choice(data['Nom del joc'])
            print(f' Perquè no jugueu a {pick} ?')
            
            # scrapeo de la imagen del juego
            object_id = None
            try:
                # scrapeo de html para obtener la id del juego
                image = requests.get(f'https://www.boardgamegeek.com/search/boardgame?q={pick}&nosession=1&showcount=20', headers = {'user-agent': '{Mozilla/5.0 (Macintosh; Intel Mac OS X 10_15_7) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/95.0.4638.69 Safari/537.36}'})
                object_pattern = re.compile('/boardgame/([0-9]*)/')
                object_id = object_pattern.findall(image.text)[0]
                
                # scrapeo de api interna a partir de la id del juego
                resp = requests.get(f'https://api.geekdo.com/api/images?ajax=1&gallery=all&nosession=1&objectid={object_id}&objecttype=thing&pageid=1&showcount=36&size=thumb&sort=hot')
                # de las Imagenes recuperados seleccionamos una al azar de las 10 primeras
                image_url = resp.json()['images'][random.randint(0,10)]['imageurl_lg']
            except:
                print("No s'ha pogut trobar cap imatge del joc\n")
                image_url = None
            
            # scrapeo de ofertas a partir del object id
            
            resp2 = None
            if object_id:
                resp2 = requests.get(f'https://api.geekdo.com/api/geekmarket/products?ajax=1&nosession=1&objectid={object_id}&objecttype=thing&pageid=1&showcount=10&stock=instock')
            else:
                print('no se ha podido encontrar el juego en la web de referencia')
            
            # si encontramos oferta , mostramos precio, currency y la url para comprar
            if resp2: 
                try:

                    for i in range(0,1):

                        for j in resp2.json()['products'][i].items():
                            if j[0] == 'price':
                                print(j[1])
                            if j[0] == 'currency':
                                print(j[1])

                            if j[0] == 'producthref':
                                print('\n Opción de compra: ')
                                url = 'https://www.boardgamegeek.com/' + j[1]
                                print(url)
                except:
                    print('no se encontraron opciones de compra para este juego')
            
            # scrapeo de reseña 

            # buscaremos en otra web "misut meeple" reseñas de los juegos
            resp3 = requests.get('https://misutmeeple.com/resenas-list/')
            
            bs_obj = BeautifulSoup(resp3.content,'html.parser')
            
            # guardamos la ruta al listado completo de reseñas para acceder a ella más fácilmente..
            tabla_rese = bs_obj.find('html').find('body').find('div', attrs={'class':'site'}).find('div', attrs={'class':'site-content'}).find('div', attrs={'class':'content-area'}).find('main').find('article').find('div').find('div').find('table')
            
            # recuperamos los juegos y las urls y guardamos en un dataframe
            rese_ls = []
            for i in tabla_rese.find('tbody'):
                if not isinstance(i.find('a'),int):
                    rese_ls.append({
                        'juego':i.find('a').text,
                        'reseña': i.find('a').get('href')
                    })
            
            rese_df = pd.DataFrame(rese_ls)
            # guardando resultados scrapping en formato csv...
            rese_df.to_csv('./my_games/reseñas.csv')
            
            # regex raro por si el nombre del juego es algo distinto
            game_search2 = pick[2:-2]
            game_patt = re.compile(f'.*{game_search2}.*')
            
            # buscamos si el juego seleccionado está en nuestro dataframe
            not_found = True
            for i in range(0,len(rese_df['juego'])):
                if pick.upper()  == (rese_df.iloc[i,0].upper()):
                    print('\n Reseña del juego: ')
                    print(rese_df.iloc[i,0])
                    print(rese_df.iloc[i,1])
                    not_found = False
            
            # probamos a buscar solo una parte del nombre del juego por regex
            try:
                if not_found:
                    for i in range(0,len(rese_df['juego'])):
                        if len(game_patt.findall(rese_df.iloc[i,0])) > 0:
                            print(rese_df.iloc[i,0])
                            print(rese_df.iloc[i,1])
                            not_found = False
                            break
            except:
                pass
            
            if not_found:
                print('no se encontró ninguna reseña para este juego')
            
            # finalmente mostrar imagen en jupyter si la hemos encontrado
            if image_url:
                try:
                    display(Image(image_url))
                except:
                    print('No es pot mostrar la imatge')
            
            #preguntar si este juego le gusta al usuario, de lo contrario hacer bucle otra vez
            agree = input("T'agrada aquesta opció? Y/N ")
            if agree.upper() == 'Y':
                break

In [3]:
class JocDB:
    
    def __init__(self):
        self.jocs_list = []
        self.jocs_df = None
        
    
    def apiCall(self):
        '''usa la api de google sheets per recuperar la informació de la meva colecció'''
        # obtenim la jocs de taula sheet desded l'Api de google

        # Entrem amb permisos de readonly
        SCOPES = ['https://www.googleapis.com/auth/spreadsheets.readonly']

        # The ID and range of a sample spreadsheet.
        SAMPLE_SPREADSHEET_ID = '1Ixtc7aIaMxe8XeKQC9fAfBerZxYFsifS0sy0X2yJfB4'
        SAMPLE_RANGE_NAME = 'Jocs - Gerard and Cris!A1:O222'

        creds = None
            # The file token.json stores the user's access and refresh tokens, and is
            # created automatically when the authorization flow completes for the first
            # time.
        if os.path.exists('token.json'):
            creds = Credentials.from_authorized_user_file('token.json', SCOPES)
        # If there are no (valid) credentials available, let the user log in.
        if not creds or not creds.valid:
            if creds and creds.expired and creds.refresh_token:
                creds.refresh(Request())
            else:
                flow = InstalledAppFlow.from_client_secrets_file(
                    'credentials.json', SCOPES)
                creds = flow.run_local_server(port=0)
            # Save the credentials for the next run
            with open('token.json', 'w') as token:
                token.write(creds.to_json())
        service = build('sheets', 'v4', credentials=creds)

        # Call the Sheets API
        print('retrieving boardgame collection from google sheets...')
        sheet = service.spreadsheets()
        result = sheet.values().get(spreadsheetId=SAMPLE_SPREADSHEET_ID,
                                    range=SAMPLE_RANGE_NAME
                                   ).execute()
        values = result.get('values', [])
        self.jocs_list = values
        
    def toDf(self):
        '''transforma la informació obtinguda en un dataframe'''
        # creem a mà un Dataframe accedint a les llistes com si fosin columnes
        self.jocs_df = pd.DataFrame({'Nom del joc':[self.jocs_list[i][1] for i in range(3,len(self.jocs_list))],
                                     'Num. Jugadors':[self.jocs_list[i][2] for i in range(3,len(self.jocs_list))],
                                      'Gènere':[self.jocs_list[i][3] for i in range(3,len(self.jocs_list))],
                                      'Millor a 2?':[self.jocs_list[i][4] for i in range(3,len(self.jocs_list))],
                                      "Dificultat d'aprenentatge (sobre 5)":[self.jocs_list[i][5] for i in range(3,len(self.jocs_list))],
                                      'Profunditat':[float(self.jocs_list[i][6]) for i in range(3,len(self.jocs_list))],
                                      'Duració':[int(self.jocs_list[i][8]) for i in range(3,len(self.jocs_list))]
                                     })
        
        # continuem tractant el df
        minjoc = re.compile('([1-9]) a [1-9]')
        maxjoc = re.compile('[1-9] a ([1-9])')
        
        # creem noves columnes amb el min i el max
        for i in self.jocs_df['Num. Jugadors'].index:
            min_p = minjoc.findall(self.jocs_df.loc[i,'Num. Jugadors'])
            if len(min_p) > 0:
                min_p = int(min_p[0])
                self.jocs_df.loc[i,'min'] = min_p
            else:
                self.jocs_df.loc[i,'min'] = 0
            max_p = maxjoc.findall(self.jocs_df.loc[i,'Num. Jugadors'])
            if len(max_p) > 0:
                max_p = int(max_p[0])
                self.jocs_df.loc[i,'max'] = max_p
            else:
                self.jocs_df.loc[i,'max'] = 0
            
        #jocs només per un numero fix de persones
        for i in self.jocs_df['Num. Jugadors'].index:
            if self.jocs_df.loc[i,'min'] == 0:
                self.jocs_df.loc[i,'min'] = int(self.jocs_df.loc[i,'Num. Jugadors'])
                self.jocs_df.loc[i,'max'] = int(self.jocs_df.loc[i,'Num. Jugadors'])
    
       


In [4]:
def main():
    # creem la conexió
    coleccio = JocDB()
    try:
        coleccio.apiCall()
    except:
        raise 'Conexió amb API ha fallat'
    
    # transformem a DataFrame
    coleccio.toDf()
    game_df = coleccio.jocs_df
    
    #export current_df to csv
    print('guardando coleccion actual en formato csv..')
    game_df.to_csv('./my_games/coleccion.csv')
    
    # creem el recomanador
    recom = Recomanador()
    recom.simple_questions()
    recom.simple_choice(game_df)

In [ ]:
main()